<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://user-images.githubusercontent.com/16768318/73986808-75b3ca00-4936-11ea-90f1-3a6c352766ce.png" width=10% >
<img align="right" style="padding-left:10px;" src="https://user-images.githubusercontent.com/16768318/73986811-764c6080-4936-11ea-9653-a3eacc47caed.png" width=10% >

**Bienvenidos!** Este *colab notebook* es parte del curso [**Introduccion a Google Earth Engine con Python**](https://github.com/csaybar/EarthEngineMasterGIS) desarrollado por el equipo [**MasterGIS**](https://www.mastergis.com/). Obten mas informacion del curso en este [**enlace**](https://www.mastergis.com/product/google-earth-engine/). El contenido del curso esta disponible en [**GitHub**](https://github.com/csaybar/EarthEngineMasterGIS) bajo licencia [**MIT**](https://opensource.org/licenses/MIT).

### **Ejercicio N°01: RUSLE a Nivel Mundial**
  <img src="https://user-images.githubusercontent.com/16768318/73690808-1604b700-46c9-11ea-8bdd-43e0e490a0a3.gif" align="right" width = 60%/>

Genere una funcion para calcular la Ecuacion Universal de Perdida de Suelo (RUSLE) para cualquier parte del mundo. La funcion debe tener los siguientes parametros.**rusle(roi, prefix, folder, scale)**

http://cybertesis.unmsm.edu.pe/handle/cybertesis/10078


In [2]:
#@title Credenciales Google Earth Engine
import ee
ee.Authenticate()
ee.Initialize(project = 'ee-jtantaroman')

In [3]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

### **1) Factor R**

El **factor R** es el factor de erosividad de la lluvia. Este factor indica el potencial erosivo de la lluvia que afecta en el proceso de erosion del suelo. Haciendo una analogia, se podria decir que una lluvia fuerte un dia al año puede producir suficiente energia para erosionar el suelo que varias lluvias de mediana intensidad a lo largo de un ano.

El factor erosividad (R) es definido como la sumatoria anual de los promedios de los valores individuales del indice de tormenta de erosion (EI30). Donde E es la energia cinetica por unidad de area e I30 es la maxima intensidad en 30 minutos de precipitacion. Esto se puede definir en la siguiente ecuacion:

<img src="https://user-images.githubusercontent.com/16768318/73694650-67fd0b00-46d0-11ea-87f6-4ed9501cf964.png"  width = 60%>

Por tanto, la energia de la tormenta (EI o R) indica el volumen de lluvia y escurrimiento, pero una larga y suave lluvia puede tener el mismo valor de E que una lluvia de corta y mas alta intensidad. (Mannaerts, 1999). La energia se calcula a partir de la formula de Brown y Foster:

<img src="https://user-images.githubusercontent.com/16768318/73694782-b3171e00-46d0-11ea-94fe-94f3f57941c5.png"  width = 40%>

A partir de la ecuación anterior, el calculo del factor R es un proceso complejo y requiere datos horarios o diarios de varios anos. Por lo que se han desarrollado diferentes ecuaciones que adaptan la erosividad local mediante una formula que solo requiera una data mensual o anual de precipitacion. A continuacion, se muestran algunas de las formulas adaptadas para una precipitacion media anual.

<img src="https://user-images.githubusercontent.com/16768318/73694993-228d0d80-46d1-11ea-8bc4-9962963850b7.png">


Si bien es cierto, se usa ampliamente una precipitacion media anual para estimar el **factor R** debido a la escasez de informacion, para este ejemplo se ha optado por  utilizar la formula desarrollada por **(Wischmeier & Smith, 1978)** debido a que se cuenta con una serie historica de informacion de precipitacion mensual. La formula es:


<img src="https://user-images.githubusercontent.com/16768318/73695488-2b321380-46d2-11ea-8033-0063f27698d8.png" width = 50%>

In [ ]:
# Monthly precipitation in mm at 1 km resolution:
# https://zenodo.org/record/3256275#.XjibuDJKiM8
clim_rainmap = ee.Image("OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01")
#Este producto tiene los datos mensuales en cada una de sus bandas por eso se hara un reduce y se sumarizara para obtener el anual
#Se aplica el reduce en la imagen de muchas bandas (todos los datos mes a mes) para obtener una sola banda
year = clim_rainmap.reduce(ee.Reducer.sum())
#R calculado mensualmente para todos los meses
R_monthly = ee.Image(10).pow(ee.Image(1.5).multiply(clim_rainmap.pow(2).divide(year).log10().subtract(-0.08188))).multiply(1.735)
#Sumamos todos los R para tener un R anual, esta es la erosividad por la lluvia
factorR = R_monthly.reduce(ee.Reducer.sum())

In [ ]:
R_monthly.bandNames().getInfo()

['jan',
 'feb',
 'mar',
 'apr',
 'may',
 'jun',
 'jul',
 'aug',
 'sep',
 'oct',
 'nov',
 'dec']

In [ ]:
#Viendo la erosividad por la lluvia
center_coordinate = [0,0]
palette_rain = ["#450155", "#3B528C", "#21918D", "#5DCA63","#FFE925"]
mapdisplay(center_coordinate, {'Factor_R':factorR.getMapId({'min':0,'max':6000,'palette':palette_rain})},zoom_start=3)

### **2) Factor K**

A diferencia del factor R, el factor K muestra qué tan susceptible es el suelo a la erosion hidrica, esto es determinado por las propiedades fisicas y quimicas del suelo, que dependen de las caracteristicas de estos. Para determinar el factor K, existen una gran cantidad de formulas empiricas, adecuadas para diversos lugares del mundo y donde intervienen caracteristicas del suelo como porcentaje de arena, limo, arcilla; estructura del suelo; contenido de carbono organico o materia orgánica; entre otros.

El factor K puede variar en una escala de 0 a 1, donde 0 indica suelos con la menor susceptibilidad a la erosion y 1 indica suelos altamente susceptibles a la erosion hidrica del suelo; cabe mencionar que esta escala fue hecha para el sistema de unidades americanas, y adaptandose al sistema internacional, la escala varia a normalmente entre 0 y 0.07.

A continuacion, se muestran algunas ecuaciones para la estimación de este factor:

<img src="https://user-images.githubusercontent.com/16768318/73704444-039b7500-46eb-11ea-9ccd-b7850bb17911.png" width = 50%>
<img src="https://user-images.githubusercontent.com/16768318/73704442-039b7500-46eb-11ea-870c-a557ca50b777.png" width = 50%>
<img src="https://user-images.githubusercontent.com/16768318/73704443-039b7500-46eb-11ea-9469-104f04983dfd.png" width = 50%>


Para este ejemplo se ha optado por  utilizar la formula desarrollada por **Williams (1975)**.



In [ ]:
# Cargamos toda la informacion necesaria para estimar el factor K
sand = ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").select('b0')
silt = ee.Image('users/aschwantes/SLTPPT_I').divide(100)
clay = ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").select('b0')
morg = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select('b0').multiply(0.58)
sn1 = sand.expression('1 - b0 / 100', {'b0': sand})
orgcar = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select('b0')

In [ ]:
#Juntando todas las imagenes en una sola
soil = ee.Image([sand, silt, clay, morg, sn1, orgcar]).rename(['sand', 'silt', 'clay', 'morg', 'sn1', 'orgcar'] )

In [ ]:
factorK = soil.expression(
  '(0.2 + 0.3 * exp(-0.0256 * SAND * (1 - (SILT / 100)))) * (1 - (0.25 * CLAY / (CLAY + exp(3.72 - 2.95 * CLAY)))) * (1 - (0.7 * SN1 / (SN1 + exp(-5.51 + 22.9 * SN1))))',
  {
    'SAND': soil.select('sand'),
    'SILT': soil.select('silt'),
    'CLAY': soil.select('clay'),
    'MORG': soil.select('morg'),
    'SN1':  soil.select('sn1'),
    'CORG': soil.select('orgcar')
  });

In [ ]:
center_coordinate = [0,0]
palette_k = palette = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]
viz_param_k = {'min': 0.0, 'max': 0.5, 'palette': palette_k};
mapdisplay(center_coordinate, {'Factor_K':factorK.getMapId(viz_param_k)},zoom_start=3)

### **3) Factor LS**

El factor LS expresa el efecto de la topografia local sobre la tasa de erosion del suelo, combinando los efectos de la longitud de la pendiente (L) y la inclinación de la pendiente (S). A medida que mayor sea la longitud de la pendiente, mayor sera la cantidad de escorrentia acumulada y de la misma forma, mientras mas pronunciada sea la pendiente de la superficie, mayor sera la velocidad de la escorrentia, que influye directamente en la erosion. Existen diversas metodologias basadas en SIG para calcular estos factores, como se pueden mostrar a continuación:

<img src="https://user-images.githubusercontent.com/16768318/73706484-7ce99680-46f0-11ea-8e0e-5fbb4a00731d.png" width = 50%>

In [ ]:
facc = ee.Image("WWF/HydroSHEDS/15ACC")
dem = ee.Image("WWF/HydroSHEDS/03CONDEM")
slope = ee.Terrain.slope(dem)

ls_factors = ee.Image([facc, slope]).rename(['facc','slope'])

In [ ]:
factorLS = ls_factors.expression(
  '(FACC*270/22.13)**0.4*(SLOPE/0.0896)**1.3',
  {
    'FACC': ls_factors.select('facc'),
    'SLOPE': ls_factors.select('slope')
  });

In [ ]:
center_coordinate = [0,0]
palette_ls = palette = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]
viz_param_k = {'min': 0, 'max': 600, 'palette': palette_ls};
mapdisplay(center_coordinate, {'Factor_LS':factorLS.getMapId(viz_param_k)},zoom_start=3)

### **4) Factor C**

El factor C se utiliza para determinar la eficacia relativa de los sistemas de manejo del suelo y de los cultivos en terminos de prevencion o reduccion de la perdida de suelo. Este factor indica como la cobertura vegetal y los cultivos afectaran la perdida media anual de suelos y como se distribuira el potencial de perdida de suelos en el tiempo (Rahaman, 2015).

El valor de C depende del tipo de vegetacion, la etapa de crecimiento y el porcentaje de cobertura. Valores mas altos del factor C indican que no hay efecto de cobertura y perdida de suelo, mientras que el menor valor de C significa un efecto de cobertura muy fuerte que no produce erosion.



In [ ]:
ndvi_median = ee.ImageCollection("MODIS/006/MOD13A2").median().multiply(0.0001).select('NDVI')

/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD13A2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13A2

  warnings.warn(warning, category=DeprecationWarning)


In [ ]:
geo_ndvi = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
]
l8_viz_params = {'palette':geo_ndvi,'min':0,'max': 0.8}
mapdisplay([0,0],{'composite_median':ndvi_median.getMapId(l8_viz_params)},zoom_start=3)

Otra forma de hallar este factor C, es haciendo una comparación entre el NDVI a partir de las fórmulas Van de Kniff (1999) [C1] y su adaptacion para paises asiaticos, que tambien se adecuan a la realidad de la costa peruana de Lin (2002) [C2]. Por ultimo se tiene la ecuacion formulada por De Jong(1994) [C3] adaptado a estudios de degradacion de suelos en un entorno mediterraneo.
<center>
<img src="https://user-images.githubusercontent.com/16768318/73713048-e6bf6b80-4703-11ea-80b1-1940e6b55707.png" width = 50%>
</center>

In [ ]:
factorC = ee.Image(0.805).multiply(ndvi_median).multiply(-1).add(0.431)

### **5) Calculo de la Erosion**

**A = R\*K\*LS\*C\*1**

  <img src="https://user-images.githubusercontent.com/16768318/73690808-1604b700-46c9-11ea-8bdd-43e0e490a0a3.gif">

In [ ]:
erosion = factorC.multiply(factorR).multiply(factorLS).multiply(factorK)

In [ ]:
geo_erosion = ["#00BFBF", "#00FF00", "#FFFF00", "#FF7F00", "#BF7F3F", "#141414"]
l8_viz_params = {'palette':geo_erosion,'min':0,'max': 6000}
mapdisplay([0,0],{'composite_median':erosion.getMapId(l8_viz_params)},zoom_start=3)

## **Ejercicio Propuesto 1**

### **Funcion para descargar RUSLE en cualquier parte del mundo**
[Respuesta aqui](https://gist.github.com/csaybar/19a9db35f8c8044448d885b68e8c9eb8)


#### Estimar la Erosión multianual para cualquier parte del mundo. El resultado final puede ser un vídeo ([ejemplo](https://drive.google.com/file/d/1u14mx7GfBj0TmDi6xTCk_vdxT0fBQN_B/view)) o enviar el código para revisarlo en formato .py o .txt.

In [14]:
#### Ponga su funcion aqui (cree un snippet!)
def rusle(roi, prefix, folder, scale):

  # Factor R
  clim_rainmap = ee.Image("OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01").clip(roi)
  year = clim_rainmap.reduce(ee.Reducer.sum())
  #R calculado mensualmente para todos los meses
  R_monthly = ee.Image(10).pow(ee.Image(1.5).multiply(clim_rainmap.pow(2).divide(year).log10().subtract(-0.08188))).multiply(1.735)
  #Sumamos todos los R para tener un R anual, esta es la erosividad por la lluvia
  factorR = R_monthly.reduce(ee.Reducer.sum()).rename('factorR')

  # Factor K
  # Cargamos toda la informacion necesaria para estimar el factor K
  sand = ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").select('b0').clip(roi)
  silt = ee.Image('users/aschwantes/SLTPPT_I').divide(100).clip(roi)
  clay = ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").select('b0').clip(roi)
  morg = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select('b0').multiply(0.58).clip(roi)
  sn1 = sand.expression('1 - b0 / 100', {'b0': sand})
  orgcar = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select('b0').clip(roi)
  #Juntando todas las imagenes en una sola
  soil = ee.Image([sand, silt, clay, morg, sn1, orgcar]).rename(['sand', 'silt', 'clay', 'morg', 'sn1', 'orgcar'] )
  factorK = soil.expression(
  '(0.2 + 0.3 * exp(-0.0256 * SAND * (1 - (SILT / 100)))) * (1 - (0.25 * CLAY / (CLAY + exp(3.72 - 2.95 * CLAY)))) * (1 - (0.7 * SN1 / (SN1 + exp(-5.51 + 22.9 * SN1))))',
  {
    'SAND': soil.select('sand'),
    'SILT': soil.select('silt'),
    'CLAY': soil.select('clay'),
    'MORG': soil.select('morg'),
    'SN1':  soil.select('sn1'),
    'CORG': soil.select('orgcar')
  }).rename('factorK');

  # Factor LS
  facc = ee.Image("WWF/HydroSHEDS/15ACC").clip(roi)
  dem = ee.Image("WWF/HydroSHEDS/03CONDEM").clip(roi)
  slope = ee.Terrain.slope(dem)
  ls_factors = ee.Image([facc, slope]).rename(['facc','slope'])
  factorLS = ls_factors.expression(
    '(FACC*270/22.13)**0.4*(SLOPE/0.0896)**1.3',
    {
      'FACC': ls_factors.select('facc'),
      'SLOPE': ls_factors.select('slope')
    }).rename('factorLS');

  # Factor C
  ndvi_median = ee.ImageCollection("MODIS/006/MOD13A2").median().multiply(0.0001).select('NDVI').clip(roi)
  factorC = ee.Image(0.805).multiply(ndvi_median).multiply(-1).add(0.431).rename('factorC')

  # Calculo de la Erosion
  erosion = factorC.multiply(factorR).multiply(factorLS).multiply(factorK).rename('A')
  erosion_output = ee.Image([erosion,factorC,factorR,factorLS,factorK])

  # Descargar RUSLE
  to_download = erosion_output
  task = ee.batch.Export.image.toDrive(
      image=to_download,
      description= prefix+'_RUSLE',
      folder=folder,
      scale=scale,
      region = roi
    )
  task.start()
  print('Descargando: '+prefix+'_RUSLE.tif')

  return erosion_output

In [15]:
# Ambito de estudio aqui
geometry = ee.Geometry.Polygon([[[-81.9580078125,-5.659718554577273],
                                [-74.99267578125,-5.659718554577273],
                                [-74.99267578125,2.04302395742204],
                                [-81.9580078125,2.04302395742204],
                                [-81.9580078125,-5.659718554577273]]])
ec_erosion = rusle(geometry,'RUSLE_','RUSLE_MASTERGIS', scale = 100)

Descargando: RUSLE__RUSLE.tif


In [16]:
ec_erosion.getInfo()

{'type': 'Image',
 'bands': [{'id': 'A',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [837, 927],
   'origin': [11765, 10238],
   'crs': 'EPSG:4326',
   'crs_transform': [0.008333333299999999,
    0,
    -179.99999999995,
    0,
    -0.008333333299999999,
    87.36999940255]},
  {'id': 'factorC',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -2.2067435,
    'max': 3.068824},
   'dimensions': [9, 10],
   'origin': [-83, -7],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'factorR',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [0.008333333299999999,
    0,
    -179.99999999995,
    0,
    -0.008333333299999999,
    87.36999940255]},
  {'id': 'factorLS',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [62870, 1853],
   'origin': [13450, 14388],
   'crs': 'EPSG:4326',
   'crs_transform': [0.004166666666667, 0, -

In [17]:
# Genere una vizualizacion de su ambito de estudio
geo_erosion = ["#00BFBF", "#00FF00", "#FFFF00", "#FF7F00", "#BF7F3F", "#141414"]
l8_viz_params = {'palette':geo_erosion,'min':0,'max': 6000}
center = geometry.centroid().coordinates().getInfo()
mapdisplay(center,{'composite_median':ec_erosion.select('A').getMapId(l8_viz_params)},zoom_start=6)

In [24]:
#### Ponga su funcion aqui (cree un snippet!)
def rusle_country(name_country):

  # Llamando al feature collection que contiene los limites de los paises
  countries = ee.FeatureCollection("USDOS/LSIB/2017")
  roi = countries.filter(ee.Filter.eq('COUNTRY_NA',name_country))
  # Calculando el centroide para la visualizacion
  center = roi.geometry().centroid().coordinates().getInfo()

  # Factor R
  clim_rainmap = ee.Image("OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01").clip(roi)
  year = clim_rainmap.reduce(ee.Reducer.sum())
  #R calculado mensualmente para todos los meses
  R_monthly = ee.Image(10).pow(ee.Image(1.5).multiply(clim_rainmap.pow(2).divide(year).log10().subtract(-0.08188))).multiply(1.735)
  #Sumamos todos los R para tener un R anual, esta es la erosividad por la lluvia
  factorR = R_monthly.reduce(ee.Reducer.sum()).rename('factorR')

  # Factor K
  # Cargamos toda la informacion necesaria para estimar el factor K
  sand = ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").select('b0').clip(roi)
  silt = ee.Image('users/aschwantes/SLTPPT_I').divide(100).clip(roi)
  clay = ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").select('b0').clip(roi)
  morg = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select('b0').multiply(0.58).clip(roi)
  sn1 = sand.expression('1 - b0 / 100', {'b0': sand})
  orgcar = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select('b0').clip(roi)
  #Juntando todas las imagenes en una sola
  soil = ee.Image([sand, silt, clay, morg, sn1, orgcar]).rename(['sand', 'silt', 'clay', 'morg', 'sn1', 'orgcar'] )
  factorK = soil.expression(
  '(0.2 + 0.3 * exp(-0.0256 * SAND * (1 - (SILT / 100)))) * (1 - (0.25 * CLAY / (CLAY + exp(3.72 - 2.95 * CLAY)))) * (1 - (0.7 * SN1 / (SN1 + exp(-5.51 + 22.9 * SN1))))',
  {
    'SAND': soil.select('sand'),
    'SILT': soil.select('silt'),
    'CLAY': soil.select('clay'),
    'MORG': soil.select('morg'),
    'SN1':  soil.select('sn1'),
    'CORG': soil.select('orgcar')
  }).rename('factorK');

  # Factor LS
  facc = ee.Image("WWF/HydroSHEDS/15ACC").clip(roi)
  dem = ee.Image("WWF/HydroSHEDS/03CONDEM").clip(roi)
  slope = ee.Terrain.slope(dem)
  ls_factors = ee.Image([facc, slope]).rename(['facc','slope'])
  factorLS = ls_factors.expression(
    '(FACC*270/22.13)**0.4*(SLOPE/0.0896)**1.3',
    {
      'FACC': ls_factors.select('facc'),
      'SLOPE': ls_factors.select('slope')
    }).rename('factorLS');

  # Factor C
  ndvi_median = ee.ImageCollection("MODIS/006/MOD13A2").median().multiply(0.0001).select('NDVI').clip(roi)
  factorC = ee.Image(0.805).multiply(ndvi_median).multiply(-1).add(0.431).rename('factorC')

  # Calculo de la Erosion
  erosion = factorC.multiply(factorR).multiply(factorLS).multiply(factorK).rename('A')
  erosion_output = ee.Image([erosion,factorC,factorR,factorLS,factorK])

  return erosion_output, center

In [33]:
ec_erosion, center = rusle_country('Venezuela')

In [34]:
# Genere una vizualizacion de su ambito de estudio
geo_erosion = ["#00BFBF", "#00FF00", "#FFFF00", "#FF7F00", "#BF7F3F", "#141414"]
l8_viz_params = {'palette':geo_erosion,'min':0,'max': 6000}
mapdisplay(center,{'composite_median':ec_erosion.select('A').getMapId(l8_viz_params)},zoom_start=6)

### **¿Dudas con este Jupyer-Notebook?**

Estaremos felices de ayudarte!. Create una cuenta Github si es que no la tienes, luego detalla tu problema ampliamente en: https://github.com/csaybar/EarthEngineMasterGIS/issues

**Tienes que dar clic en el boton verde!**

<center>
<img src="https://user-images.githubusercontent.com/16768318/79680748-d5511000-81d8-11ea-9f89-44bd010adf69.png" width = 70%>
</center>